In [3]:
from langchain.agents import Tool, initialize_agent, AgentType, load_tools
from dotenv import load_dotenv
from langchain.utilities import (
    WikipediaAPIWrapper,
    GoogleSearchAPIWrapper,
)
from langchain.tools import WikipediaQueryRun, DuckDuckGoSearchRun
from langchain import OpenAI, Wikipedia

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter,
)
from langchain.chat_models import ChatOpenAI

In [4]:
load_dotenv()

True

In [3]:
myTopic = input("Enter your topic: ")

## Getting Keywords


In [8]:
wiki = WikipediaAPIWrapper()
wikiQuery = WikipediaQueryRun(api_wrapper=wiki)
google = GoogleSearchAPIWrapper()
duck = DuckDuckGoSearchRun()

In [45]:
llm_keywords = ChatOpenAI(temperature=0.5)

In [46]:
keyword_extractor_tools = [
    Tool(
        name="Google Search",
        description="Useful when you want to get the keywords from Google about single topic.",
        func=google.run,
    ),
    Tool(
        name="DuckDuckGo Search Evaluation",
        description="Useful to evaluate the keywords of Google Search and add any missing keywords about specific topic.",
        func=duck.run,
    ),
]

In [47]:
keyword_agent = initialize_agent(
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_name="Keyword extractor",
    agent_description="You are a helpful AI that helps the user to get the important keyword list from the search results about specific topic.",
    llm=llm_keywords,
    tools=keyword_extractor_tools,
    verbose=True,
)

In [48]:
keyword_agent.agent.llm_chain.prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\nGoogle Search: Useful when you want to get the keywords from Google about single topic.\nDuckDuckGo Search Evaluation: Useful to evaluate the keywords of Google Search and add any missing keywords about specific topic.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Google Search, DuckDuckGo Search Evaluation]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [49]:
keyword_list = keyword_agent.run(
    f"Search about {myTopic} and use the results to get the important keywords related to {myTopic} to help to write a blog about {myTopic}."
)



> Entering new  chain...


I should start by searching about Mo Salah on Google to gather information about him.
Action: Google Search
Action Input: "Mo Salah"
Observation: mæd sˤɑˈlɑːħ ˈɣæːli]; born 15 June 1992), also known hypocoristically as Mo Salah, is an Egyptian professional footballer who plays as a forward for Premier ... Show more posts from mosalah. Related accounts. See all. k.mbappe. Kylian Mbappé. Follow. karimbenzema. Karim Benzema. Follow. roberto_firmino. Mohamed Salah, 31, from Egypt ➤ Liverpool FC, since 2017 ➤ Attack ➤ Market value: €65.00m ➤ * Jun 15, 1992 in Nagrig, Basyoun, Egypt. Check out the latest domestic and international stats, match logs, goals, height, weight and more for Mohamed Salah playing for Liverpool FC, ... Mohamed Salah, 31, from Egypt ➤ Liverpool FC, since 2017 ➤ Attack ➤ Market value: €65.00m ➤ * Jun 15, 1992 in Nagrig, Basyoun, Egypt. Jun 5, 2018 ... Mohamed Salah is a player who offers several glimpses into many places at once. To be honest, he is not aiming to unify

In [50]:
print(keyword_list)
# The important keywords related to Mo Salah are Mo Salah, Egyptian professional footballer, forward, Premier League, Liverpool, Egypt, market value, goals, height, weight, Liverpool FC, Champions League.

The important keywords related to Mo Salah are Mo Salah, Egyptian professional footballer, forward, Premier League, Liverpool FC, market value, goals, height, weight, Nagrig, Basyoun, Egypt, Liverpool star, Egyptian national team.


## Getting Title and SubTitle


In [35]:
title_llm = OpenAI()  # temperature=0.7

title_tools = [
    Tool(
        name="Intermediate Answer",
        description="Useful for when you need to get the title and subtitle for a blog about specific topic.",
        func=google.run,
    ),
]

self_ask_with_search = initialize_agent(
    title_tools, title_llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)
title = self_ask_with_search.run(
    f"Suggest a titel for a blog about {myTopic} using the following keywords {keyword_list}?",
)
subtitle = self_ask_with_search.run(
    f"Suggest a suitable subtitle for a blog about {myTopic} for the a blog title {title} using the following keywords {keyword_list}?",
)
print("title", title)
print("subtitle", subtitle)



> Entering new  chain...


 No.
So the final answer is: "Mo Salah - An Egyptian Professional Footballer, Liverpool FC's All-Time Top Scorer"

> Finished chain.


> Entering new  chain...
 Yes.
Follow up: What position does Mo Salah play?
Intermediate answer: Aug 3, 2022 ... Salah shared the 2021/22 Golden Boot with another player who might be described as a winger or a forward, depending on who you ask. Tottenham's ... mæd sˤɑˈlɑːħ ˈɣæːli]; born 15 June 1992), also known hypocoristically as Mo Salah, is an Egyptian professional footballer who plays as a forward for Premier ... Club: Liverpool · Nationality: Egypt · Age: 31 years (1992-06-15) · Position: Winger (Right) · Prefered foot: Left · Height: 175 cm · Weight: 71 kg · Previous teams:. Mohamed Salah, 31, from Egypt ➤ Liverpool FC, since 2017 ➤ Attack ➤ Market value: €65.00m ➤ * Jun 15, 1992 in Nagrig, Basyoun, Egypt. forward. Mohamed Salah. Buy Mohamed Salah jersey. Player highlights on LFCTV GO. Profile Stats. Mohamed Salah is one of the world's best and m

## Get search results


In [12]:
google_results = google.run(myTopic)
wiki_results = wiki.run(myTopic)
duck_results = duck.run(myTopic)
wiki_query_results = wikiQuery.run(myTopic)
print("google_results", google_results)
print("wiki_results", wiki_results)

google_results mæd sˤɑˈlɑːħ ˈɣæːli]; born 15 June 1992), also known hypocoristically as Mo Salah, is an Egyptian professional footballer who plays as a forward for Premier ... Show more posts from mosalah. Related accounts. See all. k.mbappe. Kylian Mbappé. Follow. karimbenzema. Karim Benzema. Follow. roberto_firmino. Mohamed Salah, 31, from Egypt ➤ Liverpool FC, since 2017 ➤ Attack ➤ Market value: €65.00m ➤ * Jun 15, 1992 in Nagrig, Basyoun, Egypt. Check out the latest domestic and international stats, match logs, goals, height, weight and more for Mohamed Salah playing for Liverpool FC, ... Mohamed Salah, 31, from Egypt ➤ Liverpool FC, since 2017 ➤ Attack ➤ Market value: €65.00m ➤ * Jun 15, 1992 in Nagrig, Basyoun, Egypt. Jun 5, 2018 ... Mohamed Salah is a player who offers several glimpses into many places at once. To be honest, he is not aiming to unify the world but to shrink ... May 2, 2018 ... https://www.nytimes.com/2018/05/02/world/europe/mo-salah-liverpool- ... Liverpool's Eg

## Summarise the search results


In [13]:
# summarize the results separately
summary_prompt = """Please Provide a summary of the following essay
The essay is: {essay}.
The summary is:"""
summary_prompt_template = PromptTemplate(
    template=summary_prompt,
    input_variables=["essay"],
)
summarize_llm = ChatOpenAI(
    temperature=0, model="gpt-3.5-turbo-16k"
)  # or OpenAI(temperature=0)

In [14]:
summary_chain = LLMChain(
    llm=summarize_llm,
    prompt=summary_prompt_template,
)
google_summary = summary_chain.run(essay=google_results)
wiki_summary = summary_chain.run(essay=wiki_results)
duck_summary = summary_chain.run(essay=duck_results)
wiki_query_summary = summary_chain.run(essay=wiki_query_results)

In [15]:
text_spitter = RecursiveCharacterTextSplitter(
    separators=[".", "\n", "\t", "\r", "\f", "\v", "\0", "\x0b", "\x0c"],
    chunk_size=1000,
    chunk_overlap=500,
)

In [16]:
docs = text_spitter.create_documents(
    [google_results, wiki_results, duck_results, wiki_query_results]
)
num_docs = len(docs)
summary_chain2 = load_summarize_chain(
    llm=summarize_llm,
    chain_type="map_reduce",
)

In [17]:
tot_summary = summary_chain2.run(docs)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


In [18]:
summary_tools = [
    Tool(
        name="Wikipedia",
        func=wiki.run,
        description="Search engine useful when you want to get information from Wikipedia about single topic.",
    ),
    Tool(
        name="Intermediate Answer",
        func=wikiQuery.run,
        description="Search engine useful for when you need to ask with search",
    ),
    Tool(
        name="Google Search",
        description="Search engine useful when you want to get information from Google about single topic.",
        func=google.run,
    ),
    Tool(
        name="DuckDuckGo Search",
        description="Search engine useful when you want to get information from DuckDuckGo about single topic.",
        func=duck.run,
    ),
]

In [19]:
summary_agent = initialize_agent(
    summary_tools,
    llm=summarize_llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    agent_name="Summary Agent",
    verbose=True,
)

In [20]:
tot_summary2 = summary_agent.run(
    f"can you provide me a summary about {myTopic} from each search engine separately? \ then use this information to combine all the summaries together to get a blog about {myTopic}."
)



> Entering new  chain...
I need to gather information about Mo Salah from each search engine separately. Then I can combine the summaries to create a blog about him.
Action: Wikipedia
Action Input: Mo Salah
Observation: Page: Mohamed Salah
Summary: Mohamed Salah Hamed Mahrous Ghaly (Arabic: محمد صلاح حامد محروس غالي, Egyptian Arabic pronunciation: [mæˈħam.mæd sˤɑˈlɑːħ ˈɣæːli]; born 15 June 1992), also known hypocoristically as Mo Salah, is an Egyptian professional footballer who plays as a forward for Premier League club Liverpool and captains the Egypt national team. Considered one of the best players of his generation and amongst the greatest African players of all time, he is known for his clinical finishing, dribbling and speed.Salah started his senior career in 2010 playing for Al Mokawloon, departing in 2012 to join Basel, where he won two Swiss Super League titles. In 2014, Salah joined Chelsea for a reported fee of £11 million, but limited gametime led to successive loans to 

## Create a chain to write the first draft of the blog


In [21]:
prompt_writer = """You are an expert online blogger with expert writing skills and I want you to only write out the breakdown of each section of the blog on the topic of {topic} 
using the following information:
keywords: {keywords}.
The title is: {title}.
The subtitle is: {subtitle}.
google results: {google_results}.
wiki results: {wiki_results}.
duck results: {duck_results}.
wiki query results: {wiki_query_results}.
google summary: {google_summary}.
wiki summary: {wiki_summary}.
duck summary: {duck_summary}.
wiki query summary: {wiki_query_summary}.
The results summary is: {summary}.
The final blog will be {wordCount} words long, with an introduction at the beginning and a conclusion at the end of the blog.
use the following template to write the blog:
[BLOG]
[title]
[subtitle]
[introduction]
[body in bullet points]
[summary]
[END BLOG]
"""

prompt_writer_template = PromptTemplate(
    template=prompt_writer,
    input_variables=[
        "topic",
        "title",
        "subtitle",
        "google_results",
        "wiki_results",
        "duck_results",
        "wiki_query_results",
        "google_summary",
        "wiki_summary",
        "duck_summary",
        "wiki_query_summary",
        "summary",
        "keywords",
        "wordCount",
    ],
)
writer_llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")

In [22]:
writer_chain = LLMChain(
    llm=writer_llm,
    prompt=prompt_writer_template,
    verbose=True,
)

In [23]:
draft1 = writer_chain.run(
    topic=myTopic,
    title=title,
    subtitle=subtitle,
    google_results=google_results,
    wiki_results=wiki_results,
    duck_results=duck_results,
    wiki_query_results=wiki_query_results,
    google_summary=google_summary,
    wiki_summary=wiki_summary,
    duck_summary=duck_summary,
    wiki_query_summary=wiki_query_summary,
    summary=tot_summary + tot_summary2,
    keywords=keyword_list,
    wordCount=3000,
)



> Entering new  chain...
Prompt after formatting:
You are an expert online blogger with expert writing skills and I want you to only write out the breakdown of each section of the blog on the topic of Mo Salah 
using the following information:
keywords: The important keywords related to Mo Salah are Mo Salah, Egyptian professional footballer, forward, Premier League, Liverpool, Egypt national team, market value, Nagrig, Basyoun, attack, right winger, left foot, Liverpool FC, hattrick, Champions League..
The title is: "The Phenomenal Mo Salah: An Egyptian Professional Footballer's Journey from Nagrig to Liverpool FC".
The subtitle is: "Mo Salah: The Egyptian Professional Footballer Who Revolutionized the Premier League and the Egypt National Team".
google results: mæd sˤɑˈlɑːħ ˈɣæːli]; born 15 June 1992), also known hypocoristically as Mo Salah, is an Egyptian professional footballer who plays as a forward for Premier ... Show more posts from mosalah. Related accounts. See all. k.mbap

In [24]:
print(draft1)

[BLOG]
"The Phenomenal Mo Salah: An Egyptian Professional Footballer's Journey from Nagrig to Liverpool FC"
"Mo Salah: The Egyptian Professional Footballer Who Revolutionized the Premier League and the Egypt National Team"

[Introduction]
- Introduce Mo Salah as an Egyptian professional footballer who plays as a forward for Premier League club Liverpool FC and captains the Egypt national team.
- Mention his market value, his achievements, and his impact on Liverpool and Egypt.

[Body]
1. Early Life and Career
- Born on June 15, 1992, in Nagrig, Basyoun, Egypt.
- Started his senior career in 2010 playing for Al Mokawloon.
- Joined Basel in 2012 and won two Swiss Super League titles.
- Joined Chelsea in 2014 but had limited gametime and went on successive loans to Fiorentina and Roma.
- Signed permanently for Roma in 2016 and was a key figure in their title bid.

2. Arrival at Liverpool FC
- Signed for Liverpool in 2017 for a then-club record transfer fee.
- Set the record for most Premi

In [51]:
draft1_word_count = len(draft1.split())
print("draft1_word_count", draft1_word_count)
print(
    len(
        """[BLOG]
The Rise of Mo Salah: A Footballing Phenomenon
The Rise of the Phenomenon: Mo Salah's Journey to Success

[Introduction]
Mohamed Salah, also known as Mo Salah, is an Egyptian professional footballer who has taken the football world by storm. Playing as a forward for Premier League club Liverpool FC, Salah has become a household name and a true footballing phenomenon. In this blog, we will delve into the rise of Mo Salah and explore his journey to success.

[Body]
- Early Life and Career:
  - Salah was born on June 15, 1992, in Nagrig, Basyoun, Egypt.
  - He started his senior career in 2010, playing for Al Mokawloon.
  - In 2012, Salah joined Basel, where he won two Swiss Super League titles.
  - Salah's talent caught the attention of Chelsea, and he joined the club in 2014 for a reported fee of £11 million.
  - Limited playing time at Chelsea led to successive loans to Fiorentina and Roma, where he later signed permanently for €15 million.
  
- Success at Liverpool:
  - In 2017, Salah signed for Liverpool for a then-club record transfer of £36.9 million.
  - In his first season with Liverpool, Salah set the record for most Premier League goals scored (32) in a 38-game season.
  - He helped Liverpool reach the 2018 UEFA Champions League Final and formed a formidable attacking trio with Roberto Firmino and Sadio Mané.
  - Salah played a crucial role in Liverpool's Champions League and Premier League title successes in the following two seasons.
  - He has also won the FA Cup and the League Cup with Liverpool.
  
- Individual Accolades:
  - Salah has achieved numerous individual accolades throughout his career.
  - He has won two PFA Players' Player of the Year awards and three Premier League Golden Boots.
  - Salah has been named the Premier League Player of the Season and the Premier League Playmaker of the Season.
  - He finished third for the Best FIFA Men's Player in 2018 and 2021.
  - Salah received the 2018 FIFA Puskás Award for his winning strike in the first Merseyside derby of the 2017–18 season.
  
- International Success:
  - Salah has represented Egypt at both youth and senior levels.
  - He was named the CAF Most Promising African Talent of the Year after his performances at the 2012 Summer Olympics.
  - Salah finished as the runner-up in the 2017 and 2021 Africa Cup of Nations.
  - He was the top scorer during CAF qualification as Egypt qualified for the 2018 FIFA World Cup.
  - Salah has been named the CAF African Footballer of the Year and the BBC African Footballer of the Year multiple times.
  
- Off the Field Impact:
  - Salah is not only a footballing superstar but also a symbol of national pride in Egypt.
  - He has been named in Time magazine's list of the 100 most influential people in 2019.
  - Salah is known as the 'Fourth Pyramid' to Egyptians, and he has raised the profile of Liverpool among Egyptians.
  - He is favored across the Arab world and is often referred to as the 'Pride of the Arabs'.
  
[Summary]
Mohamed Salah's rise to success is nothing short of remarkable. From his early days in Egypt to becoming a key player for Liverpool FC, Salah has achieved greatness both individually and as part of a team. His impact on and off the field has made him a true footballing phenomenon and a source of pride for Egyptians and fans around the world.

[END BLOG]""".split()
    )
)

draft1_word_count 452
584


In [26]:
# Put the text in draft1 in text file
with open("draft1.txt", "w") as f:
    f.write(draft1)

In [52]:
def count_words_with_bullet_points(input_string):
    bullet_points = [
        "*",
        "-",
        "+",
        ".",
    ]  # define the bullet points to look for
    words_count = 0
    for bullet_point in bullet_points:
        input_string = input_string.replace(
            bullet_point, ""
        )  # remove the bullet points
    words_count = len(input_string.split())  # count the words
    return words_count

In [53]:
my_string = """[BLOG]
The Rise of Mo Salah: A Footballing Phenomenon
The Rise of the Phenomenon: Mo Salah's Journey to Success

[Introduction]
Mohamed Salah, also known as Mo Salah, is an Egyptian professional footballer who has taken the football world by storm. Playing as a forward for Premier League club Liverpool FC, Salah has become a household name and a true footballing phenomenon. In this blog, we will delve into the rise of Mo Salah and explore his journey to success.

[Body]
- Early Life and Career:
  - Salah was born on June 15, 1992, in Nagrig, Basyoun, Egypt.
  - He started his senior career in 2010, playing for Al Mokawloon.
  - In 2012, Salah joined Basel, where he won two Swiss Super League titles.
  - Salah's talent caught the attention of Chelsea, and he joined the club in 2014 for a reported fee of £11 million.
  - Limited playing time at Chelsea led to successive loans to Fiorentina and Roma, where he later signed permanently for €15 million.
  
- Success at Liverpool:
  - In 2017, Salah signed for Liverpool for a then-club record transfer of £36.9 million.
  - In his first season with Liverpool, Salah set the record for most Premier League goals scored (32) in a 38-game season.
  - He helped Liverpool reach the 2018 UEFA Champions League Final and formed a formidable attacking trio with Roberto Firmino and Sadio Mané.
  - Salah played a crucial role in Liverpool's Champions League and Premier League title successes in the following two seasons.
  - He has also won the FA Cup and the League Cup with Liverpool.
  
- Individual Accolades:
  - Salah has achieved numerous individual accolades throughout his career.
  - He has won two PFA Players' Player of the Year awards and three Premier League Golden Boots.
  - Salah has been named the Premier League Player of the Season and the Premier League Playmaker of the Season.
  - He finished third for the Best FIFA Men's Player in 2018 and 2021.
  - Salah received the 2018 FIFA Puskás Award for his winning strike in the first Merseyside derby of the 2017–18 season.
  
- International Success:
  - Salah has represented Egypt at both youth and senior levels.
  - He was named the CAF Most Promising African Talent of the Year after his performances at the 2012 Summer Olympics.
  - Salah finished as the runner-up in the 2017 and 2021 Africa Cup of Nations.
  - He was the top scorer during CAF qualification as Egypt qualified for the 2018 FIFA World Cup.
  - Salah has been named the CAF African Footballer of the Year and the BBC African Footballer of the Year multiple times.
  
- Off the Field Impact:
  - Salah is not only a footballing superstar but also a symbol of national pride in Egypt.
  - He has been named in Time magazine's list of the 100 most influential people in 2019.
  - Salah is known as the 'Fourth Pyramid' to Egyptians, and he has raised the profile of Liverpool among Egyptians.
  - He is favored across the Arab world and is often referred to as the 'Pride of the Arabs'.
  
[Summary]
Mohamed Salah's rise to success is nothing short of remarkable. From his early days in Egypt to becoming a key player for Liverpool FC, Salah has achieved greatness both individually and as part of a team. His impact on and off the field has made him a true footballing phenomenon and a source of pride for Egyptians and fans around the world.

[END BLOG]"""
word_count = count_words_with_bullet_points(my_string)
print(word_count)  # Output: 6

555


## Evaluate the first draft


In [1]:
from langchain.experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner,
)

In [5]:
# evaluation agent
evaluation_llm = ChatOpenAI(temperature=0.5, model="gpt-3.5-turbo-16k")

evaluation_prompt = """You are an expert blogs editor and you will evaluate the first the draft of the blog
in terms of the following criteria:
1- The blog must be relevant to {topic}.
2- The blog must contain the following keywords: {keywords}.
3- The blog must contain the following title: {title}.
4- The blog must contain the following subtitle: {subtitle}.
5- The blog must contain at least {wordCount} words.

Edit the first draft using the given tools to satisfy the above criteria.
The draft is:
{draft1}
The answer is the second draft which is the edited version of the first draft.
"""

In [9]:
summary_tools = [
    Tool(
        name="Wikipedia",
        func=wiki.run,
        description="Use it when you want to find an article, essay , or a blog about specific topic",
    ),
    Tool(
        name="Intermediate Answer",
        func=WikipediaQueryRun(api_wrapper=wiki).run,
        description="Use it when you want to search about specific topic inside an article, essay , or a blog.",
    ),
    Tool(
        name="Google Search",
        description="Search engine useful when you want to get information from Google about single topic in general.",
        func=google.run,
    ),
    Tool(
        name="DuckDuckGo Search",
        description="Search engine useful when you want to get information from DuckDuckGo about single topic in general.",
        func=duck.run,
    ),
]
planner = load_chat_planner(evaluation_llm, system_prompt=evaluation_prompt)
executor = load_agent_executor(evaluation_llm, summary_tools, verbose=True)
evaluation_agent = PlanAndExecute(planner=planner, executor=executor, verbose=True)

In [13]:
draft2 = evaluation_agent(
    inputs=[{
        "topic": "Mo Salah Career",
        "keywords": "Some important keywords related to Mo Salah's career are: Africa Cup of Nations, top scorer, FIFA Puskás Award, Liverpool FC, Swiss Super League, Premier League, goals, assists, African Footballer of the Year, individual goals, hat-trick, national and international career, Europa League, Basel, Tottenham Hotspur, Chelsea.",
        "title": "A Star is Born: The Story of Mo Salah's National and International Career",
        "subtitle": "From Africa Cup of Nations to Puskás Award Winner: The Story of Mo Salah's National and International Career",
        "wordCount": 2000,
        "draft1": """As an experienced writer and author, I have conducted thorough research on the best way to write a blog about Mo Salah's career. Mo Salah, an Egyptian professional footballer, has made a significant impact both nationally and internationally, playing as a forward for Premier League club Liverpool and captaining the Egypt national team. His achievements and recognition as one of the best players of his generation and among the greatest African players of all time cannot be overlooked. Salah's skills in finishing, dribbling, and speed have set him apart from his peers.

Starting his senior career in 2010 with Al Mokawloon in Egypt, Salah quickly caught the attention of Swiss side Basel, where he joined in 2012. During his time with Basel, he won two Swiss Super League titles, showcasing his talent and potential. In 2014, Salah made a move to Chelsea for a reported fee of £11 million. Although he had loan spells at Fiorentina and Roma, it was with Roma that he signed permanently, leaving a lasting impression on the Italian club.

Salah's career took a significant turn when he signed for Liverpool in 2017 for a then-club record transfer of £36.9 million. In his first season with Liverpool, he set the record for the most Premier League goals scored (32) in a 38-game season, displaying his incredible goal-scoring abilities. He played a crucial role in helping Liverpool reach the 2018 UEFA Champions League Final and formed a formidable attacking trio with Roberto Firmino and Sadio Mané.

At Liverpool, Salah achieved great success, contributing to the club's Champions League and Premier League title victories. His individual accolades include the PFA Players' Player of the Year and the Premier League Golden Boot. In 2018, he received the prestigious FIFA Puskás Award for his winning strike in the Merseyside derby, further solidifying his status as a top-class player.

On the international stage, Salah represented Egypt at various levels before making his senior debut in 2011. He finished as the runner-up in the 2017 and 2021 Africa Cup of Nations, showcasing his impact and importance to the national team. Salah's top-scoring performances during CAF qualification led Egypt to qualify for the 2018 FIFA World Cup. His outstanding contributions earned him the titles of CAF African Footballer of the Year and BBC African Footballer of the Year.

Beyond his achievements on the field, Salah has become a symbol of national pride in Egypt, earning the nickname 'Fourth Pyramid.' His success has also raised the profile of Liverpool among Egyptians and across the Arab world. In 2019, he was named in Time magazine's list of the 100 most influential people, highlighting his impact beyond the realm of football.

In summary, Mo Salah's career is a testament to his exceptional skills, dedication, and the impact he has made in the world of football. From his early days in Egypt and success in the Swiss Super League to his influential move to Liverpool, Salah has consistently impressed with his goal-scoring abilities. His contributions to Liverpool's triumphs in the Champions League and Premier League have solidified his status as one of the club's greatest players. On the international stage, Salah has represented Egypt with distinction and has been recognized as one of Africa's finest footballers. His impact extends beyond the field, as he is regarded as a symbol of national pride and has raised the profile of Liverpool among Egyptians and the Arab world. Overall, Mo Salah's career is a testament to his talent, dedication, and the impact he has made in the world of football.

Word count: 498 words""",
    },"Mo Salah"],
    include_run_info=True,
)
print(draft2)



> Entering new  chain...
steps=[]

IndexError: list index out of range